In [2]:
#Implementation of algorithm in "Haemers' conjecture: algorithmic perspective"

from itertools import product, combinations
from sage.rings.finite_rings.integer_mod_ring import IntegerModRing
from sage.graphs.graph import Graph
import gc
from signal import alarm

def compute_walk_matrix(G, e):
    """
    Compute the walk matrix W for graph G with control vector e.
    Returns W (n x n matrix, columns A^k e for k=0 to n-1).
    """
    n = G.order()
    A = G.adjacency_matrix()
    W_cols = []
    current = e
    for k in range(n):
        W_cols.append(current)
        current = A * current
    return matrix(ZZ, n, n, W_cols).transpose()


def adjust_almost_controllable(W, n, rk):
    if rk != n - 1:
        return W
    W_new = matrix(ZZ, W)  # Copy
    B = W_new[:, :n-1]  # n x (n-1)
    # Compute left kernel over QQ
    B_qq = B.change_ring(QQ)
    kernel_basis = B_qq.kernel().basis()
    v_rat = kernel_basis[0]  # 1D kernel
    # Scale by a consistent factor
    for t in range(n):
        if v_rat[t] != 0:
            lastentry = B[[i for i in range(n) if i != t], :].det()
            factor = lastentry / v_rat[t] / 2 ** (n // 2 - 1)
            v_rat *= factor
            break 
    W_new.set_column(n-1, v_rat)
    return W_new


def compute_prime_list(Delta, d):
    """
    Compute list of odd primes p where p^2 divides Delta.
    Returns list of primes pL.
    """
    if d == 0:
        return []  # Handle gcd=0 case: no common primes

    timeout_seconds = 600
    try:
        alarm(timeout_seconds)          
        f_d = d.factor()            # try to factor
        alarm(0)                        
        return [p for p, exp in f_d if p > 2 and (Delta % (p ** 2) == 0)]
    except AlarmInterrupt:              
        alarm(0)                        
        print(f"Factoring took more than {timeout_seconds} seconds → skipped")
        return None                 
    except Exception as e:
        alarm(0)
        raise e

def compute_Mp_p2(chi, n):
    """
    Compute M_p(x) for p=2.
    Returns coefficients CF and degree s.
    """
    coeffs = chi.coefficients(sparse=False)
    coeffs = [0] * (n + 1 - len(coeffs)) + coeffs
    if n % 2 == 0:
        high_coeffs = [coeffs[n - 2*i] for i in range((n//2) + 1)]
        CF = [ZZ(c) % 2 for c in high_coeffs[::-1]]
    else:
        high_coeffs = [coeffs[n - 2*i] for i in range(n//2 + 1)]
        inner_coeffs = high_coeffs[::-1]
        CF = [ZZ(0)] + [ZZ(c) % 2 for c in inner_coeffs]
    s = len(CF) - 1
    return CF, s

def compute_Mp_odd_p(A, p, n):
    """
    Compute M_p(x) for odd prime p.
    Returns coefficients CF and degree s.
    """
    Fp = GF(p)
    A_fp = A.change_ring(Fp)
    J_fp = matrix(Fp, n, n, 1)
    chi_fp = A_fp.charpoly()
    g_fp = (A_fp + J_fp).charpoly()
    Phi = gcd(chi_fp, g_fp)
    F_factors = Phi.squarefree_decomposition()
    sfp = prod(f for f, m in F_factors)
    mpoly = chi_fp // sfp
    CF = [ZZ(c) for c in mpoly.coefficients(sparse=False)]
    while len(CF) < mpoly.degree() + 1:
        CF = [ZZ(0)] + CF
    s = mpoly.degree()
    return CF, s

def compute_Wp(W, CF, s, p, rk, n):
    """
    Compute modified walk matrix W^{(p)}.
    Returns Wtemp.
    """
    Wtemp = matrix(ZZ, W)  # Create a new matrix copy
    cols = Wtemp.columns()
    for t in range(s, rk):
        lin_comb = sum(CF[j] * cols[j+t-s] for j in range(s+1))
        lin_comb_list = [x // p for x in lin_comb]
        Wtemp.set_column(t, lin_comb_list)
    if rk == n - 1:
        exp_p = n - 1 - s
        if exp_p < 0:
            exp_p = 0
        if p == 2:
            k = n // 2 - 1
            exp = max(exp_p, k) - k
        else:
            exp = exp_p
        if exp > 0:
            div = p ** exp
            last_col = [x // div for x in Wtemp.column(n-1)]
            Wtemp.set_column(n-1, last_col)
    return Wtemp

def compute_pLimproved_Wall(A, chi, W, rk, n, pL):
    """
    Compute pLimproved and Wall by checking rank of W^{(p)}.
    Returns pLimproved, Wall.
    """
    pLimproved = []
    Wall = []
    CF, s = compute_Mp_p2(chi, n)
    Wtemp = compute_Wp(W, CF, s, 2, rk, n)
    if Wtemp.change_ring(GF(2)).rank() < n:
        pLimproved.append(2)
        Wall.append(Wtemp)
    for p in pL:
        CF, s = compute_Mp_odd_p(A, p, n)
        Wtemp = compute_Wp(W, CF, s, p, rk, n)
        if Wtemp.change_ring(GF(p)).rank() < n:
            pLimproved.append(p)
            Wall.append(Wtemp)
    return pLimproved, Wall

def solve_congruences(W_p, p, m, n, A):
    """
    Solve congruence system for ξ mod m for prime p.
    Returns list of solutions pVex.
    """
    UB = 65536
    MAX_NOS = 10**6
    timeout_seconds = 600
    R_m = IntegerModRing(m)

    divisors = W_p.elementary_divisors()
   
    reduced_Wp = W_p.change_ring(R_m)
    W_p = matrix(ZZ, reduced_Wp)  # Create a new matrix copy with reduced entries
    try:
        alarm(timeout_seconds)                    
        S, U, V = W_p.transpose().smith_form()       
        alarm(0)                        
    except AlarmInterrupt:
        alarm(0)   
        gc.collect()                     
        print(f"Smith normal form took more than 600 seconds → skipped")
        return [] 
    except Exception as e:
        alarm(0)
        raise e

    gc.collect()  # Force garbage collection after smith_form

    dn = divisors[n-1]
    T = []
    for k in range(n-1, -1, -1):
        #g = gcd(m, S[k, k])
        g = gcd(m, divisors[k])
        if g > 1:
            T.append(g)
        else:
            break
    T = T[::-1]
    lenT = len(T)
    nos = prod(T)
    if nos > MAX_NOS:
        print(f"Warning: Skipping p={p} due to large enumeration size {nos} > {MAX_NOS}")
        return []  # Return dummy solution to indicate skip
    if nos > UB:
        print(f"Note: Enumeration for p={p} has size {nos} > {UB}, proceeding.")
    pVex = []
    diag_mat = diagonal_matrix(ZZ, [m // T[i] for i in range(lenT)])
    V_right = V[:, -lenT:]
   
    # List all solutions pv
    for v_tup in product(*[range(t) for t in T]):
        v = vector(ZZ, v_tup)
        pv_zz = V_right * diag_mat * v
        pv = vector(R_m, pv_zz)
        if pv * pv == 0 and pv * (A * pv) == 0:
            pVex.append(vector(ZZ, pv))
    return pVex

def compute_L_Pii(A, pLimproved, Wall, n):
    """
    Compute L, mList, and Pii (solutions ξ for each p).
    Returns L, mList, Pii.
    """
    L = 1
    mList = []
    Pii = []
    for j, p in enumerate(pLimproved):
        W_p = Wall[j]
        try:
            S = W_p.elementary_divisors()
        except MemoryError:
            print(f"Warning: MemoryError computing elementary divisors for p={p}, skipping.")
            return None, None, None # Return None to indicate failure
        dn = S[n-1]

        gc.collect()  # Force garbage collection after elementary_divisors
        if dn == 0:
            dn = compute_walk_matrix(Graph(A), vector(ZZ, [1]*n)).elementary_divisors[n-1]
            print(f"Note: For p={p}, dn=0 (infinite valuation), Use d_n(W). W_p = {W_p}, A = {A}, dn recomputed = {dn}")
            
        m = p ** valuation(dn, p)
        L *= m
        mList.append(m)
        pVex = solve_congruences(W_p, p, m, n, A)
        if len(pVex) == 0:
            print(f"Warning: No solutions found for p={p}, skipping.")
            return None, None, None # Return None to indicate failure
        Pii.append(pVex)
    return L, mList, Pii

def adjust_Pv(Pv, L, n, A, e):
    """
    Adjust Pv to satisfy x.x = L^2, e.x = L, xAx = 0.
    Returns list of adjusted vectors.
    """
    UB = 65536
    s = Pv * Pv
    total_sum = sum(Pv)
    if s > L**2 or abs(total_sum - L) > 3*L:
        return []
    d = (total_sum - L) // L
    if d not in range(-3, 4):
        return []
    adjust = {
        -3: [[3,0]],
        -2: [[2,0]],
        -1: [[1,0], [2,1]],
        0: [[0,0], [1,1]],
        1: [[0,1], [1,2]],
        2: [[0,2]],
        3: [[0,3]]
    }[d]
    indneg = [i for i in range(n) if Pv[i] < 0]
    indpos = [i for i in range(n) if Pv[i] > 0]
    indnz = indneg + indpos
    if len(indnz) == 0:
        #We have Pv = 0, which is already accounted for
        return []
    A1 = A.matrix_from_rows_and_columns(indnz, indnz)
    result = []
    for inc, dec in adjust:
        for comb_inc in combinations(range(len(indneg)), inc):
            for comb_dec in combinations(range(len(indpos)), dec):
                sum_a = sum(-Pv[indneg[k]] for k in comb_inc)
                sum_b = sum(Pv[indpos[k]] for k in comb_dec)
                right_side = (s // L + inc + dec - 1)// 2
                if sum_a + sum_b == right_side:
                    Pv1 = vector(ZZ, [Pv[i] for i in indnz])
                    for k in comb_inc:
                        Pv1[k] += L
                    for k in comb_dec:
                        Pv1[len(indneg) + k] -= L
                    if Pv1 * (A1 * Pv1) == 0:
                        Pv1n = vector(ZZ, [0]*n)
                        for ii, idx in enumerate(indnz):
                            Pv1n[idx] = Pv1[ii]
                        result.append(Pv1n)
    return result

def compute_PVex(L, mList, Pii, n, A, e):
    """
    Compute PVex using CRT and adjustments.
    Returns PVex.
    """
    UB = 6553600
    PVex = []
    for j in range(n):
        vec = vector(ZZ, [0]*n)
        vec[j] = L
        PVex.append(vec)
    if len(Pii) == 0:
        return PVex
    M = [L // m for m in mList]
    y = []
    for k in range(len(Pii)):
        m_k = mList[k]
        M_k = M[k]
        if gcd(M_k, m_k) != 1:
            print(f"Warning: Non-coprime M_k={M_k}, m_k={m_k}, gcd={gcd(M_k, m_k)}")
            raise ValueError(f"Non-invertible M[{k}]={M_k} mod mList[{k}]={m_k}: gcd={gcd(M_k, m_k)}")
        y_k = ZZ(inverse_mod(M_k, m_k))
        y.append(y_k)
    TT = [len(pii) for pii in Pii]
    Nos = prod(TT)
    if Nos > UB:
        print(f"Note: CRT enumeration size {Nos} > {UB}, computationally expensive not proceeding.")
        return []
    if Nos == 0:
        return []
    for v_tup in product(*[range(tt) for tt in TT]):
        Pv = vector(ZZ, [0]*n)
        for k in range(len(Pii)):
            Pv += Pii[k][v_tup[k]] * y[k] * M[k]
        for i in range(n):
            Pv[i] = Pv[i] % L
            if Pv[i] > (L-1)//2:
                Pv[i] -= L
        PVex.extend(adjust_Pv(Pv, L, n, A, e))
    return PVex

def construct_omega_G(PVex, L, A, n):
    """
    Construct the graph Omega(G).
    Returns graph qG.
    """
    num_v = len(PVex)
    AG = matrix(ZZ, num_v, num_v, 0)
    for t1 in range(num_v):
        for t2 in range(t1+1, num_v):
            xy = PVex[t1] * PVex[t2]
            xAy = PVex[t1] * (A * PVex[t2])
            if xy == 0 and (xAy == 0 or xAy == L**2):
                AG[t1, t2] = 1
                AG[t2, t1] = 1
    return Graph(AG)

def is_DS(G):
    """
    Determines if graph G is DS (determined by spectrum) using Algorithm 1.
    Assumes G is controllable or almost controllable.
    Returns True if DS, False otherwise.
    """
    n = G.order()
    if n == 0:
        return 4  # Empty graph case
    A = G.adjacency_matrix()
    e = vector(ZZ, [1] * n)
    R = PolynomialRing(QQ, 'x')
    chi = A.charpoly()
    Delta = chi.discriminant()

    # Compute walk matrix and check controllability
    W = compute_walk_matrix(G, e)
    rk = W.rank()
    if rk < n - 1:
        return -1 # Not controllable or almost controllable
    W = adjust_almost_controllable(W, n, rk)
    Deter = W.det()
    d = gcd(Delta, Deter)

    # Compute pLimproved and Wall
    pL = compute_prime_list(Delta, d)
    if pL is None:
        return -3  # Factoring failed or timed out
    pLimproved, Wall = compute_pLimproved_Wall(A, chi, W, rk, n, pL)
    if len(pLimproved) == 0:
        return 1

    # Compute L, Pii, and PVex
    L, mList, Pii = compute_L_Pii(A, pLimproved, Wall, n)
    if L is None:
        return -2 #Either memory issues in elementary_divisors or a prime factor was too big in solve_congruences

    PVex = compute_PVex(L, mList, Pii, n, A, e)
    
    if len(PVex) == 0:
        return -4 #Could not combine solutions  in CRT.
        
    # Check if Omega(G) has n vertices
    if len(PVex) == n:
        return 1

    # Construct Omega(G) and find cliques
    qG = construct_omega_G(PVex, L, A, n)

    lln = list(sage.graphs.cliquer.all_cliques(qG,min_size = n, max_size = n))

    # Check DS condition
    num_cliques = len(lln)
    if num_cliques == 1:
        return 2
    if rk == n - 1 and num_cliques == 2:
        aut_group = G.automorphism_group()
        if aut_group.order() == 1:
            return 3
    return 0

In [5]:
# Run the algorithm on the 2-core of the giant component of sparse graphs

from collections import Counter

def giant_component(G):
    '''Returns the largest connected component of G.'''
    components = G.connected_components()
    largest = max(components, key=len)
    return G.subgraph(largest)

def two_core(G):
    '''Returns the 2-core of G.'''
    H = G.copy()
    changed = True
    while changed:
        changed = False
        for v in H.vertices():
            if H.degree(v) < 2:
                H.delete_vertex(v)
                changed = True
                break
    return H

pari.allocatemem(2^33)

#Choose the order of the graph and the average degree
n = 50
deglist = [1.2, 1.5, 2, 2.5, 5]

generated_graphs = []

for edges in deglist:
    temp = []
    for _ in range(100):
        temp.append(two_core(giant_component(graphs.RandomGNP(n, edges/(n-1)))))
    generated_graphs.append(temp)

ds = []
order = []
gstrings = []
countedDS = []
for edges in range(len(deglist)):   
    deg = deglist[edges]
    print('average degree:', deg)
    gc.collect()

    order.append(sum(G.order() for G in generated_graphs[edges]))
    temp = []

    with open(f'Wangalg_results_degree_{deg}_{n}.txt', 'w') as f:
        for G in generated_graphs[edges]:
            wangalg_output = is_DS(G)
            f.write(f'{wangalg_output} ! {G.sparse6_string()} \n')
            temp.append(wangalg_output)
    
    ds.append(temp)

    countedDS.append(Counter(ds[edges]))

print(order)
print(countedDS)


PARI stack size set to 8589934592 bytes, maximum size set to 8589934592
average degree: 1.20000000000000
average degree: 1.50000000000000
Note: Enumeration for p=383 has size 146689 > 65536, proceeding.
Note: Enumeration for p=257 has size 66049 > 65536, proceeding.
average degree: 2
Note: Enumeration for p=2 has size 131072 > 65536, proceeding.
Note: Enumeration for p=383 has size 146689 > 65536, proceeding.
average degree: 2.50000000000000
Note: Enumeration for p=2 has size 524288 > 65536, proceeding.
Note: Enumeration for p=2 has size 524288 > 65536, proceeding.
Note: Enumeration for p=2 has size 131072 > 65536, proceeding.
Note: Enumeration for p=2 has size 131072 > 65536, proceeding.
average degree: 5
Note: CRT enumeration size 7030400 > 6553600, computationally expensive not proceeding.
Note: Enumeration for p=2 has size 131072 > 65536, proceeding.
[441, 959, 2386, 3271, 4834]
[Counter({4: 49, -1: 48, 1: 3}), Counter({-1: 47, 4: 25, 1: 24, -2: 2, 2: 1, 3: 1}), Counter({1: 65, -1: